# [o3]- Proyecto Ozono - ETL_Contaminación (2001-202004) - v3

    0. Inicializacion
    1. Datos
        1.0 Carga ficheros AIRE ( 2001-2020) - URL
        1.1 FORMATO:  [ESTACION,MAGNITUD,ANO,MES,DATO_DIA1,VALIDO,DATO_DIA2,VALIDO...]
        1.2 Creacion nuevo esquema
        1.3 Creacion nuevo DF vacio
        1.4 Nuevo DF -> [ESTACION,MAGNITUD,ANO,MES,DIA,FECHA,VALOR,VALIDO]
        1.5 Columna FECHA -> 20140101
        1.6 Colocar columnas -> [ESTACION,MAGNITUD, ANO,MES,DIA, FECHA, VALOR,VALIDO]
        1.7 VALOR ? VALIDO -> VALOR VALIDADO
        1.8 PIVOT Magnitudes
        1.9 Limpiar: Meses 31 días + Meses 30 días + Febreros
    2. Formato
    3. Ordenar
    4. Exportar
        
    

# [0] - Inicialización

In [17]:
from __future__ import print_function
import findspark
findspark.init('/home/rulicering/BigData/spark-2.4.5-bin-hadoop2.7')
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField,StringType,IntegerType,FloatType,StructType
from pyspark.ml.feature import StringIndexer
from pyspark.sql import functions as F
import pandas as pd
import datetime
import requests

In [18]:
spark = SparkSession.builder.appName('contaminacion').getOrCreate()

# [1] - Datos

## [1.0] - Carga ficheros AIRE ( 2014-2020) - URL

In [19]:
anos= (2001,2020)
urls =["https://datos.madrid.es/egob/catalogo/201410-10306604-calidad-aire-diario.csv",
       "https://datos.madrid.es/egob/catalogo/201410-10306602-calidad-aire-diario.csv",
       "https://datos.madrid.es/egob/catalogo/201410-10306600-calidad-aire-diario.csv",
       "https://datos.madrid.es/egob/catalogo/201410-10306598-calidad-aire-diario.csv",
       "https://datos.madrid.es/egob/catalogo/201410-10306596-calidad-aire-diario.csv",
       "https://datos.madrid.es/egob/catalogo/201410-10306594-calidad-aire-diario.csv",
       "https://datos.madrid.es/egob/catalogo/201410-10306592-calidad-aire-diario.csv",
       "https://datos.madrid.es/egob/catalogo/201410-10306590-calidad-aire-diario.csv",
       "https://datos.madrid.es/egob/catalogo/201410-10306588-calidad-aire-diario.csv",
       "https://datos.madrid.es/egob/catalogo/201410-10306586-calidad-aire-diario.csv",
       "https://datos.madrid.es/egob/catalogo/201410-10306584-calidad-aire-diario.csv",
       "https://datos.madrid.es/egob/catalogo/201410-10306582-calidad-aire-diario.csv",
       "https://datos.madrid.es/egob/catalogo/201410-10306580-calidad-aire-diario.csv",
       "https://datos.madrid.es/egob/catalogo/201410-10306578-calidad-aire-diario.csv",
       "https://datos.madrid.es/egob/catalogo/201410-10306576-calidad-aire-diario.csv",
       "https://datos.madrid.es/egob/catalogo/201410-10306574-calidad-aire-diario.csv",
       "https://datos.madrid.es/egob/catalogo/201410-7775098-calidad-aire-diario.csv",
       "https://datos.madrid.es/egob/catalogo/201410-7775096-calidad-aire-diario.csv",
       "https://datos.madrid.es/egob/catalogo/201410-10306606-calidad-aire-diario.csv",
       "https://datos.madrid.es/egob/catalogo/201410-10306609-calidad-aire-diario.csv"  
]

In [20]:
def pd_read_to_df(url):
    pdf = pd.read_csv(url,sep=';')
    df = spark.createDataFrame(pdf)
    return df

In [21]:
magnitudes_aire = ["1","6","7","8","9","10","12","14","20","30","35","42","43","44"]

In [22]:
def procesoxano(anos):
    l_pd = []
    for i in range(anos[1] - anos[0]+1):
        print(anos[0]+i)
        df = pd_read_to_df(urls[i])
        # [1.1] - FORMATO:  [ESTACION,MAGNITUD,ANO,MES,DATO_DIA1,VALIDO,DATO_DIA2,VALIDO...]
        #Eliminamos columnas no esenciales
        df = df.select('ESTACION','MAGNITUD','ANO','MES','D01','V01','D02','V02','D03','V03','D04','V04','D05','V05','D06','V06','D07','V07','D08','V08','D09','V09','D10','V10','D11','V11','D12','V12','D13','V13','D14','V14','D15','V15','D16','V16','D17','V17','D18','V18','D19','V19','D20','V20','D21','V21','D22','V22','D23','V23','D24','V24','D25','V25','D26','V26','D27','V27','D28','V28','D29','V29','D30','V30','D31','V31')
        # [1.2] - Creacion nuevo esquema
        data_schema = [StructField('ESTACION',IntegerType(), False), 
              StructField('MAGNITUD',IntegerType(), False),
              StructField('ANO',IntegerType(), False),
              StructField('MES',IntegerType(), False),
              StructField('VALOR',FloatType(), True),
              StructField('VALIDO',IntegerType(), False),
              StructField('DIA',IntegerType(), False)]
        struct = StructType(fields = data_schema)
        # [1.3] - Creacion nuevo DF vacio
        df_v1 = spark.createDataFrame(spark.sparkContext.emptyRDD(),struct)
        # [1.4] - Nuevo DF -> [ESTACION,MAGNITUD,ANO,MES,DIA,FECHA,VALOR,VALIDO]
        for i in range(1,32): #Días  
            valor = 'D%02d' % i
            valido = 'V%02d' % i
            df_v1 = df_v1.union(df.select("ESTACION","MAGNITUD","ANO","MES",valor,valido).withColumn('DIA', F.lit(i)))

        df = df_v1
        # [1.5] - Columna FECHA -> 20140101
        df = df.withColumn("FECHA",df["ANO"]*10000 + df["MES"]*100 + df["DIA"])
        # [1.6] -Colocar columnas -> [ESTACION, MAGNITUD, ANO,MES,DIA, FECHA, VALOR,VALIDO]
        cols = df.columns
        cols = cols[:4] + cols[-2:] + cols[-4:-2]
        df= df[cols]
        # [1.7] - VALOR ? VALIDO -> VALOR VALIDADO
        df = df.withColumn("VALOR VALIDADO",F.when(F.col("VALIDO")== 'N',None).otherwise(F.col("VALOR")) )
        df = df.select("ESTACION","MAGNITUD","ANO","MES","DIA","FECHA",df["VALOR VALIDADO"].alias("VALOR"))
        ## [1.8] - PIVOT Magnitudes
        df = df.groupBy('ESTACION','ANO', 'MES', 'DIA',"FECHA").pivot("MAGNITUD").sum("VALOR").orderBy("FECHA")
        ## [1.9] - Limpiar: Meses 31 días + Meses 30 días + Febreros
        #Esto se puede hacer en el paso 6
        df_31 = df.filter(df["MES"].isin([1,3,5,7,8,10,12]))
        df_30 = df.filter((df["MES"].isin([4,6,9,11])) & (df["DIA"] <31))
        df_feb = df.filter((df["MES"] == 2) & (df["DIA"] <30)) #Para no excluir los bisiestos
        df = df_31.union(df_30).union(df_feb)
        # [2] - Formato
        df = df.withColumn("ESTACION",df["ESTACION"].cast(StringType()))
        df = df.withColumnRenamed("ESTACION","CODIGO_CORTO")
        #Estandarizar el numero de columnas. Las que no existen, a nulo
        cols_magnitudes = df.columns[5:]
        for magnitud in magnitudes_aire:
            if(magnitud not in cols_magnitudes):
                df = df.withColumn(magnitud,F.lit(None))
        #Ordenamos las columnas
        cols_magnitudes = df.columns[5:]
        cols_magnitudes.sort()
        cols = df.columns[0:5] + cols_magnitudes
        df = df.select(cols)
        #Lo pasamos a pandas y lo añadimos a la lista (Por motivos de capacidad computacional)
        pd_df = df.toPandas()
        l_pd.append(pd_df)

    #Unimos los dfs de cada año
    pd_final = pd.concat(l_pd)
    return pd_final

In [23]:
pd = procesoxano(anos)

2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


# [3] - Ordenar 

In [24]:
#df = df.sort("FECHA",ascending= True )

# [4] - Exportar

In [25]:
pd_final = pd

In [26]:
#Versiones
hoy = datetime.date.today().strftime("%Y-%m-%d")
pd_final.to_csv("/home/rulicering/Datos_Proyecto_Ozono/Procesado/Contaminacion/BackUp/Contaminacion_diaria_2001-202004-" + hoy +".csv")

In [27]:
pd_final.to_csv("/home/rulicering/Datos_Proyecto_Ozono/Procesado/Contaminacion/Contaminacion_diaria_2001-202004.csv")